In [1]:
#예제 1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
jack = Customer('Park', 100)
jack

Customer(name='Park', fidelity=100)

In [54]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product  
        self.quantity = quantity  
        self.price = price 

    def total(self):
        return self.price * self.quantity 

class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출 """
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

class FidelityPromo(Promotion): 
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0


In [55]:
# 예제 2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]


In [56]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [57]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [58]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [59]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [60]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [61]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [62]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [63]:
# 예제 3
class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


In [64]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [65]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [66]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [67]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [8]:
# 예쩨 4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """C최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [9]:
# 예제 5
print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


In [10]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'jack', '_2', '_i3', 'LineItem', 'Order', 'Promotion', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i4', 'joe', 'ann', 'cart', 'banana_cart', 'long_order', '_i5', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i6', '_i7', '_i8', 'promos', 'best_promo', '_i9', '_i10'])

In [11]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [12]:
# 예제 6
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

promos


[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [ ]:
### 07.Function Decorators and Closures

In [15]:
# 예제 1
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

print(target())
print(target)

running inner()
None
<function deco.<locals>.inner at 0x000001C3D95B9760>


In [16]:
registry = []  # 여기에 decorated function 추가

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x000001C3D95B8AE0>)
running register(<function f2 at 0x000001C3D95B89A0>)
running main()
registry -> [<function f1 at 0x000001C3D95B8AE0>, <function f2 at 0x000001C3D95B89A0>]
running f1()
running f2()
running f3()


In [17]:
# 예제 3
promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)


In [18]:
# 예제 4
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [19]:
b = 6
f1(3)

3
6


In [20]:
# 예제 5
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [23]:
# 예제 6
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
6


In [24]:
b

9

In [25]:
f3(3)

3
9


In [26]:
b=30
f3(3)

3
30


In [28]:
# 예제 7
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)


In [29]:
avg = Averager()
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

In [32]:
# 예제 8
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [33]:
avg = make_averager()
avg(10)

10.0

In [34]:
avg(11)

10.5

In [35]:
avg(12)

11.0

In [36]:
# 예제 9
avg.__code__.co_varnames

('new_value', 'total')

In [37]:
avg.__code__.co_freevars

('series',)

In [38]:
avg.__closure__

(<cell at 0x000001C3D90EB610: list object at 0x000001C3D95E5740>,)

In [39]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [42]:
# 예제 10
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [43]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [44]:
# 예제 11
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [45]:
avg = make_averager()
avg(10)

10.0

In [46]:
# 예제 12
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [47]:
# 예제 13
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[0.12308800s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000100s] factorial(1) -> 1
[0.00003950s] factorial(2) -> 2
[0.00006320s] factorial(3) -> 6
[0.00008700s] factorial(4) -> 24
[0.00010960s] factorial(5) -> 120
[0.00013340s] factorial(6) -> 720
6! = 720


In [49]:
# 예제 14
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))



**************************************** Calling snooze(.123)
[0.12710094s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720


In [50]:
factorial.__name__

'factorial'